<a href="https://colab.research.google.com/github/maxysio/DS-PRJ2-DataStructures/blob/master/05-BlockChain.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import hashlib
from time import gmtime, strftime
import time

In [0]:
def calc_hashx(data):
    sha = hashlib.sha256()
    sha.update(data.encode('utf-8'))
    return sha.hexdigest()

In [0]:
class Block:
    def __init__(self, timestamp, data, previous_hash, current_hash):
      self.timestamp = timestamp
      self.data = data
      self.previous_hash = previous_hash
      self.hash = current_hash

In [0]:
class BlockChain:
  def __init__(self):
    self.blocks = []
    self.head = None
    
  def calc_hash(self, data):
    sha = hashlib.sha256()
    sha.update(data.encode('utf-8'))
    return sha.hexdigest()

  def prepend(self, data):
    # Create a new block
    data_hash = self.calc_hash(data)
    new_block = Block(time.gmtime(), data, None, data_hash)
    self.blocks.insert(0, new_block)

    if self.head is None:
      # If no head, that would mean its an empty BlockChain
      self.head = new_block
      return
    
    # Assign the new blocks hash to the previous hash of the head and move head to the new block
    self.head.previous_hash = new_block.hash
    self.head = new_block

  def append(self, data):
    # Create a new block
    data_hash = self.calc_hash(data)
    new_block = Block(time.gmtime(), data, None, data_hash)
    self.blocks.append(new_block)

    if self.head is None:
      # If no head, that would mean no tail, assign both to new block and return
      self.head = new_block
      return

    # Assign the hash from the tail to the new blocks previous hash, move the tail to the new block
    new_block.previous_hash = self.calc_hash(self.blocks[-1].data)

  def search(self, data):
    # Run through the list to find the block object
    for b in self.blocks:
      if b.data == data:
        return b
    print('{} not found in BlockChain'.format(data))
    return None
    
  def remove(self, data):
    # If the size is 0, then nothing to remove
    if self.size() == 0:
      return
    
    block = self.search(data)
    if block:
      # If data is found in the block chain,
      # Check if the size of the block chain is 1, then its the only object
      if self.size() == 1:
        self.blocks = []
        return

      # Find the block before and after it and link them
      
      current_block_hash = self.calc_hash(block.data)
      prev_block = None
      next_block = None
      
      # For the block before, travel through the list, 
      # check block's previous hash against hash of the data in the block in the list
      for b in self.blocks:
        if block.previous_hash == self.calc_hash(b.data):
          prev_block = b
          break
      
      # For the block after, travel through the list,
      # check hash of current block's data against the previous hash of the block in the list
      for b in self.blocks:
        if b.previous_hash == current_block_hash:
          next_block = b
          break
      
      if next_block and prev_block:
        print('Here')
        # Block to be deleted in the middle
        next_block.previous_hash = self.calc_hash(prev_block.data)
      elif next_block:
        # Block to be deleted is in the begining
        next_block.previous_hash = None

      # If Block to be deleted is at the end, no action item

      # Finally remove the block from the list
      self.blocks.remove(block)

  def pop(self):
    # Remove the last item from the Block Chain
    
    # Check if the block chain is empty
    if self.size()==0:
      print('BlockChain is empty')
      return

    block = self.blocks[-1]

    # Remove
    self.remove(block.data)

    return block

  def insert(self, data, pos):
    # if block chain is empty, just add it irrespective of position
    if self.size() == 0:
      self.append(data)
      return

    # if pos is 1, then its the begining, so prepend it
    if pos == 1:
      self.prepend(data)
      return

    # if position is greater than the size of the BlockChain, add it to the end
    if pos > self.size():
      self.append(data)
      return

    # Create a new block
    data_hash = self.calc_hash(data)
    new_block = Block(time.gmtime(), data, None, data_hash)

    # Get the block from the list for that position and the previous one
    next_block = self.blocks[pos-1]
    prev_block = self.blocks[pos-2]

    # assign the has correctly
    new_block.previous_hash = next_block.previous_hash
    next_block.previous_hash = data_hash
    
    # insert new block into the list
    self.blocks.insert(pos-1, new_block)

  def size(self):
    return len(self.blocks)

In [0]:
bc = BlockChain()
bc.append('Block1')
bc.append('Block2')
bc.append('Block3')
bc.append('Block4')

In [131]:
bc.size()

4

In [135]:
for b in bc.blocks:
  print(b.data, 'Current Block Hash: {}'.format(calc_hashx(b.data)), 'Prev Hash: {}'.format(b.previous_hash))

Block67 Current Block Hash: 721984ef2b862d17a1e9329e46f8eaa85a70de9e88c248872e5a259c6264dca3 Prev Hash: None
Block1 Current Block Hash: 40e9b17a3391b5f461b2b96a2e5810a885f088346b901c65ebb5cf8cf7361103 Prev Hash: 721984ef2b862d17a1e9329e46f8eaa85a70de9e88c248872e5a259c6264dca3
Block2 Current Block Hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380 Prev Hash: 61edd5d6b03c20f764aab3bc4291b162ff48958e316603d4c07548a37872e380
Block3 Current Block Hash: f76000270122d79ba262f8298080d37b8645d471d3885999274deba5caa7f704 Prev Hash: f76000270122d79ba262f8298080d37b8645d471d3885999274deba5caa7f704
Block4 Current Block Hash: ca174e0d60e2740ce87f5d4b303104edbaa26eae3792f2dc9711e2ea527d6b3c Prev Hash: ca174e0d60e2740ce87f5d4b303104edbaa26eae3792f2dc9711e2ea527d6b3c


In [0]:
bc.insert('Block67', 1)

In [134]:
bc.head.data

'Block67'